In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests

In [2]:
# scrape list of NBA teams from Wikipedia

teams = []

r = requests.get("https://en.wikipedia.org/wiki/National_Basketball_Association")
text = r.text
soup = BeautifulSoup(text, 'html.parser')
rows = soup.find("table", attrs={"class": "navbox wikitable"}).find_all("td")
for row in rows:
    elements = row.find_all("b")
    for el in elements:
        teams.append(el.text)
#printing to check        
print len(teams)

30


In [3]:
teams

[u'Boston Celtics',
 u'Brooklyn Nets',
 u'New York Knicks',
 u'Philadelphia 76ers',
 u'Toronto Raptors',
 u'Chicago Bulls',
 u'Cleveland Cavaliers',
 u'Detroit Pistons',
 u'Indiana Pacers',
 u'Milwaukee Bucks',
 u'Atlanta Hawks',
 u'Charlotte Hornets',
 u'Miami Heat',
 u'Orlando Magic',
 u'Washington Wizards',
 u'Denver Nuggets',
 u'Minnesota Timberwolves',
 u'Oklahoma City Thunder',
 u'Portland Trail Blazers',
 u'Utah Jazz',
 u'Golden State Warriors',
 u'Los Angeles Clippers',
 u'Los Angeles Lakers',
 u'Phoenix Suns',
 u'Sacramento Kings',
 u'Dallas Mavericks',
 u'Houston Rockets',
 u'Memphis Grizzlies',
 u'New Orleans Pelicans',
 u'San Antonio Spurs']

In [4]:
# download ticket data into a dataframe
count = 0
for team in teams:
    team_lower = team.lower()
    team_final = team_lower.replace(" ", "+")
    r = requests.get('http://api.seatgeek.com/2/events?q=' + team_final + '&per_page=1000')
    js = r.json()
    events = js["events"]

    df = pd.DataFrame()

    if count == 0:
        for event in events:
            keys =  event.keys()
            break

    for key in keys:
        lst = []
        listing_count = []
        avg_price = []
        lowest_price_good_deal = []
        lowest_price = []
        highest_price = []
        address = []
        city = []
        country = []
        display_location = []
        extended_address = []
        venueid = []
        location = []
        venuename = []
        postal_code = []
        venuescore = []
        venueslug = []
        state = []
        timezone = []
        url = []
       
        for event in events:
            if key == "stats":
                listing_count.append(event[key]["listing_count"])
                avg_price.append(event[key]["average_price"])
                lowest_price_good_deal.append(event[key]["lowest_price_good_deals"])
                lowest_price.append(event[key]["lowest_price"])
                highest_price.append(event[key]["highest_price"])
            elif key == "venue":
                address.append(event[key]["address"])
                city.append(event[key]["city"])
                country.append(event[key]["country"])
                display_location.append(event[key]["display_location"])
                extended_address.append(event[key]["extended_address"])
                venueid.append(event[key]["id"])
                location.append(event[key]["location"])
                venuename.append(event[key]["name"])
                postal_code.append(event[key]["postal_code"])
                venuescore.append(event[key]["score"])
                venueslug.append(event[key]["slug"])
                state.append(event[key]["state"])
                timezone.append(event[key]["timezone"])
                url.append(event[key]["url"])
            else:
                lst.append(event[key])
        if key == "stats":
            df["listing_count"] = listing_count
            df["avg_price"] = avg_price
            df["lowest_price_good_deal"] = lowest_price_good_deal
            df["lowest_price"] = lowest_price
            df["highest_price"] = highest_price
        elif key == "venue":
            df["address"] = address
            df["city"] = city
            df["country"] = country
            df["display_location"] = display_location
            df["extended_address"] = extended_address
            df["venueid"] = venueid
            df["location"] = location
            df["venuename"] = venuename
            df['postal_code'] = postal_code
            df["venuescore"] = venuescore
            df["venueslug"] = venueslug
            df["venuestate"] = state
            df["venuetimezone"] = timezone
            df['url'] = url
        else:
            df[key] = lst

    if count == 0:
        fulldf = df
    else:
        fulldf = fulldf.append(df, ignore_index = True)
    count = count + 1

In [5]:
# clean the data

# rename score as ticketscore to distinguish it from other scores
fulldf=fulldf.rename(columns = {'score':'ticket_score'})

# make sure you only have nba games
fulldf = fulldf.loc[fulldf['type'] == "nba"]
# get rid of the all-star game
fulldf = fulldf.loc[fulldf['title'] != "NBA All-Star Game"]

# drop unnecessary columns
fulldf = fulldf.drop("time_tbd", 1)
fulldf = fulldf.drop("taxonomies", 1)
fulldf = fulldf.drop("links", 1)
fulldf = fulldf.drop("url", 1)
fulldf = fulldf.drop("display_location", 1)
fulldf = fulldf.drop("venueslug", 1)
fulldf = fulldf.drop("venuetimezone", 1)
fulldf = fulldf.drop("date_tbd", 1)
fulldf = fulldf.drop("type", 1)
fulldf = fulldf.drop("location", 1)

# sources: http://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

In [6]:
fulldf.head()

,listing_count,avg_price,lowest_price_good_deal,lowest_price,highest_price,title,datetime_local,created_at,performers,address,city,country,extended_address,venueid,venuename,postal_code,venuescore,venuestate,announce_date,short_title,datetime_utc,ticket_score,datetime_tbd,visible_until_utc,id
1,673,91,38,38,623,Washington Wizards at Boston Celtics,2015-11-27T19:30:00,2015-08-13T00:00:00,"[{u'home_team': True, u'stats': {u'event_count...",100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85462,MA,2015-08-13T00:00:00,Wizards at Celtics,2015-11-28T00:30:00,0.78197,False,2015-11-28T04:30:00,2759719
3,347,48,14,14,1351,Boston Celtics at Orlando Magic,2015-11-29T18:00:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",400 West Church Street,Orlando,US,"Orlando, FL 32801",2652,Amway Center,32801,0.76076,FL,2015-08-13T00:00:00,Celtics at Magic,2015-11-29T23:00:00,0.71332,False,2015-11-30T03:00:00,2759739
4,2309,65,16,16,3548,Boston Celtics at Miami Heat,2015-11-30T19:30:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",601 Biscayne Blvd,Miami,US,"Miami, FL 33132",183,American Airlines Arena,33132,0.85785,FL,2015-08-13T00:00:00,Celtics at Heat,2015-12-01T00:30:00,0.80130,False,2015-12-01T04:30:00,2759732
6,1340,139,28,28,1266,Boston Celtics at San Antonio Spurs,2015-12-05T19:30:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",1 AT&T Center,San Antonio,US,"San Antonio, TX 78219",185,AT&T Center,78219,0.76469,TX,2015-08-13T00:00:00,Celtics at Spurs,2015-12-06T01:30:00,0.76659,False,2015-12-06T05:30:00,2759759
7,1485,34,6,6,813,Boston Celtics at New Orleans Pelicans,2015-12-07T19:00:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",1501 Girod Street,New Orleans,US,"New Orleans, LA 70113",184,Smoothie King Center,70113,0.75731,LA,2015-08-13T00:00:00,Celtics at Pelicans,2015-12-08T01:00:00,0.73714,False,2015-12-08T05:00:00,2759738


In [18]:
# put teams in separate columns

teams_playing = fulldf.title
away_teams = []
home_teams = []
for teams_pl in teams_playing:
    split = teams_pl.split(" at ")
    away_teams.append(split[0])
    home_teams.append(split[1])
    
fulldf["home_team"] = home_teams
fulldf["away_team"] = away_teams
fulldf = fulldf.drop("title", 1)

AttributeError: 'DataFrame' object has no attribute 'title'

In [19]:
# unpack performers dicts

performers = fulldf.performers
home_teams = list(fulldf["home_team"])
away_teams = list(fulldf["away_team"])

hometeamscore = []
awayteamscore = []
count = 0
lens = []
for perf in performers:
    lens.append(len(perf))
    for p in perf:
        if p.keys()[0] == "home_team":
            hometeamscore.append(p["score"])
        elif p.keys()[0] == "away_team":
            awayteamscore.append(p["score"])
    count = count + 1

fulldf["hometeamscore"] = hometeamscore
fulldf["awayteamscore"] = awayteamscore

fulldf = fulldf.drop("performers", 1)
# sources: http://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe

AttributeError: 'DataFrame' object has no attribute 'performers'

In [20]:
# calculate days away

import time
import datetime

datetimes = fulldf["datetime_utc"]
date = []
timeutc = []
daysaway = []
toddate = datetime.datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d").date()

for dattime in datetimes:
    futdate = datetime.datetime.strptime(dattime.split("T")[0],"%Y-%m-%d").date()
    timeaway = futdate - toddate
    daysaway.append(int(timeaway.days))
    date.append(dattime.split("T")[0])
    timeutc.append(dattime.split("T")[1])
fulldf["dateutc"] = date
fulldf["timeutc"] = timeutc
fulldf["daysaway"] = daysaway

# sources: http://www.cyberciti.biz/faq/howto-get-current-date-time-in-python/

In [21]:
# reset index
fulldf = fulldf.reset_index()

# save as a CSV
#fulldf.to_csv("/Users/baekstboy/Documents/Senior/Stat121/ActiveFPData/basketballdf" + str(toddate) + ".csv")

# sources: http://chrisalbon.com/python/pandas_saving_dataframe_as_csv.html

ValueError: cannot insert level_0, already exists

In [22]:
# A little EDA:
# plt.scatter(fulldf["daysaway"], fulldf["lowest_price"])

# print np.mean(fulldf["lowest_price"][fulldf["daysaway"] < 90])
# print np.mean(fulldf["lowest_price"][fulldf["daysaway"] > 90])
#     # the hole at about 80 days is the NBA All Star Game

In [23]:
# OUTLINE:
# Short time horizon:
    # logistic for probability that appreciates more than x%
    # regression with y = final price after x days
# In order to extend to further time horizons without data, can regress today's price
    # get ceof. on days away
    # forecast by changing days away

In [24]:
fulldf.head()

,level_0,index,listing_count,avg_price,lowest_price_good_deal,lowest_price,highest_price,datetime_local,created_at,address,city,country,extended_address,venueid,venuename,postal_code,venuescore,venuestate,announce_date,short_title,datetime_utc,ticket_score,datetime_tbd,visible_until_utc,id,home_team,away_team,hometeamscore,awayteamscore,dateutc,timeutc,daysaway
0,0,1,673,91,38,38,623,2015-11-27T19:30:00,2015-08-13T00:00:00,100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85462,MA,2015-08-13T00:00:00,Wizards at Celtics,2015-11-28T00:30:00,0.78197,False,2015-11-28T04:30:00,2759719,Boston Celtics,Washington Wizards,0.74910,0.71482,2015-11-28,00:30:00,2
1,1,3,347,48,14,14,1351,2015-11-29T18:00:00,2015-08-13T00:00:00,400 West Church Street,Orlando,US,"Orlando, FL 32801",2652,Amway Center,32801,0.76076,FL,2015-08-13T00:00:00,Celtics at Magic,2015-11-29T23:00:00,0.71332,False,2015-11-30T03:00:00,2759739,Orlando Magic,Boston Celtics,0.67888,0.74912,2015-11-29,23:00:00,3
2,2,4,2309,65,16,16,3548,2015-11-30T19:30:00,2015-08-13T00:00:00,601 Biscayne Blvd,Miami,US,"Miami, FL 33132",183,American Airlines Arena,33132,0.85785,FL,2015-08-13T00:00:00,Celtics at Heat,2015-12-01T00:30:00,0.80130,False,2015-12-01T04:30:00,2759732,Miami Heat,Boston Celtics,0.78556,0.74910,2015-12-01,00:30:00,5
3,3,6,1340,139,28,28,1266,2015-12-05T19:30:00,2015-08-13T00:00:00,1 AT&T Center,San Antonio,US,"San Antonio, TX 78219",185,AT&T Center,78219,0.76469,TX,2015-08-13T00:00:00,Celtics at Spurs,2015-12-06T01:30:00,0.76659,False,2015-12-06T05:30:00,2759759,San Antonio Spurs,Boston Celtics,0.70488,0.74910,2015-12-06,01:30:00,10
4,4,7,1485,34,6,6,813,2015-12-07T19:00:00,2015-08-13T00:00:00,1501 Girod Street,New Orleans,US,"New Orleans, LA 70113",184,Smoothie King Center,70113,0.75731,LA,2015-08-13T00:00:00,Celtics at Pelicans,2015-12-08T01:00:00,0.73714,False,2015-12-08T05:00:00,2759738,New Orleans Pelicans,Boston Celtics,0.64740,0.74910,2015-12-08,01:00:00,12


In [25]:
fulldf.shape[0]

2006